In [ ]:
# This notebook is meant to run on the nlpcluster. 
# See instructions https://www.notion.so/Helpers-9ef437d130c9484c9eb5be38f6d041a2 to launch a notebook on the cluster
# Reminder: Use the conda environment 'gptj'

In [ ]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/nlp/scr/shibani/gpt/models'
from transformers import GPTJForCausalLM
import torch

model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", revision="float16", 
                                         torch_dtype=torch.float16, low_cpu_mem_usage=True
                                       ).cuda()
from transformers import AutoModelForCausalLM, AutoTokenizer
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
model = model.cuda()

In [ ]:
from copy import deepcopy

labels = ['land','water','air']
labels_shuffled = deepcopy(labels)


# The different types of animals
land=['donkey', 'goat', 'cat', 'dog', 'lion', 
                  'tiger', 'elephant', 'giraffe', 'wolf', 'pig',
                 'sheep','rat','mice','horse','cow']
water=['dolphin', 'whale', 'orca', 'turtle', 'seal', 'octopus', 
                'crab', 'sea lion', 'otter', 'seahorse', 
               'starfish', 'squid', 'lobster','eel','shark']
air=['bat', 'eagle', 'owl', 'vulture', 'hummingbird', 'albatross', 
                   'pegion', 'crow', 'flamingo', 'falcon', 'fowl', 
                    'pelican', 'sparrow', 'crane','stork']



# Add data to a dictionary
animals = {}
animals['land'] = land
animals['water'] = water
animals['air'] = air

In [ ]:
import numpy as np
import random
from numpy.random import randint

# Perform a seeded runs below. Set the parameters for the seeds here.
num_seeds=6            
seeds=np.ndarray.tolist(np.arange(0,num_seeds,step=1))

# List of training fraction of the data 
train_ratio_list = [0.5, 0.6, 0.7, 0.8, 0.9]    

test_errors = []
truelabel_test_errors = []
shuffle_labels = True        # Set to True to permute the labels associated with the data


for seed in seeds:
    test_errors.append([])
    truelabel_test_errors.append([])
    print('Current seed is', seed)
    random.seed(seed)
    for train_ratio in train_ratio_list:
        print('Train ratio is', train_ratio)
        
        if shuffle_labels==True:
            shuffle=0
            while shuffle==0:
                shuffle = 1
                random.shuffle(labels_shuffled)   # Shuffles the list of labels
                for i in range(len(labels_shuffled)):
                    if labels_shuffled[i] == labels[i]:
                        shuffle=0

        train_data = []
        test_data = []
        test_label = []
        true_test_label = []
        
        # Generate training and test data
        for i in range(len(labels)):
            group_full = animals[labels[i]]
            group_train = random.sample(group_full,int(len(group_full)*train_ratio))
            group_test = list(set(group_full) - set(group_train))
            for j in range(len(group_train)):
                train_point = group_train[j]+'-'+labels_shuffled[i]
                train_data.append(train_point)
            for j in range(len(group_test)):
                test_point = group_test[j]+'-'
                test_data.append(test_point)
                test_label.append(labels_shuffled[i])
                true_test_label.append(labels[i])

            random.shuffle(train_data)  # Permute the train data so that all the groups dont appear together

        # Create the prompts
        prompts = []
        train_prompt = ''
        for i in range(len(train_data)):
            train_prompt = train_prompt+train_data[i]+'\n'
        for i in range(len(test_data)):
            test_prompt = train_prompt+test_data[i]
            prompts.append(test_prompt)


        # Feed the prompts to GPT-3
        completions = []
        prediction = []
        num_errors = 0
        true_errors = 0
        for i in range(len(prompts)):
            input_ids = tokenizer(prompts[i], return_tensors="pt").input_ids
            output = model.generate(input_ids.cuda(), do_sample=True, 
                                    temperature=0.5, max_length=len(prompt.split(' ')) + 30,)
            completions.append(output)
            output_label = tokenizer.batch_decode(output)[0].replace(prompts[i],'')
            prediction.append(output_label)
            prediction.append(completions[i].choices[0].text)
            num_errors += int(prediction[i]!=test_label[i])
            true_errors += int(prediction[i]!=true_test_label[i])

        test_error = num_errors/len(test_label)
        test_errors[seed].append(test_error)
        truelabel_test_errors[seed].append(true_errors/len(test_label))

In [ ]:
test_errors_np = np.array(test_errors)
mean_error = np.mean(test_errors_np,axis=0)
std_error = np.std(test_errors_np,axis=0)

#Error with respect to true labels
true_label_error_np = np.array(truelabel_test_errors)
mean_true_error = np.mean(true_label_error_np,axis=0)
std_true_error = np.std(true_label_error_np,axis=0)


print('Error:',mean_error,'Std dev.:',std_error)
print('True Label Error:',mean_true_error,'Std dev.:',std_true_error)

In [ ]:
#Generate plot with the loss results
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()
number_prompts = (3*15*np.array(train_ratio_list)).astype(int)

plt.plot(number_prompts, mean_error,label='Error')
plt.plot(number_prompts,mean_true_error,label='True Label Error')
plt.fill_between(number_prompts, (mean_error-0.68*std_error), 
                         (mean_error+0.68*std_error), 
                             color='blue', alpha=0.1)
plt.fill_between(number_prompts, (mean_true_error-0.68*std_true_error), 
                         (mean_true_error+0.68*std_true_error), 
                             color='red', alpha=0.1)

plt.xlabel('Number of prompt examples')
plt.ylabel('Error')
plt.legend(loc='best')
plt.title('Learning random permutations')
plt.savefig('../../Results/permutation_error_gpt3',format='pdf')